<a href="https://colab.research.google.com/github/ykkim77/nlp-14th/blob/main/RNN_poet_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import os
import numpy as np
import sys
import tensorflow as tf
from sklearn.model_selection import train_test_split
from korean import compose, decompose

import matplotlib.pyplot as plt

### 데이터 가져오기

In [ ]:
def read_poets(poet_dir):
    """시가 담겨져 있는 디렉토리 내에서 시 내용들을 가져오는 함수
    :param poet_dir : *.txt 포맷으로 저장된 시가 담긴 디렉토리 위치
    
    :return 
        list of the poet(type: str)
    """
    poets = []
    for poet_name in os.listdir(poet_dir):
        poet_path = os.path.join(poet_dir, poet_name)
        with open(poet_path,'r') as f:
            poets.append(f.read())
    return poets

In [ ]:
# 백석 시를 읽어옮
poets = read_poets("poets/")

In [ ]:
len(poets) # 시는 총 10편, 모두 백석 시인의 작품

10

In [ ]:
poets[0] # 나와 나타샤와 흰 당나귀

'가난한 내가\n아름다운 나타샤를 사랑해서 \n오늘밤은 푹푹 눈이 나린다 \n나타샤를 사랑은 하고 \n눈은 푹푹 날리고 \n나는 혼자 쓸쓸히 앉어 소주를 마신다 \n소주를 마시며 생각한다 \n나타샤와 나는 눈이 푹푹 쌓이는 밤\n흰 당나귀 타고 산골로 가자 출출이 우는\n깊은 산골로 가 마가리에 살자 \n눈은 푹푹 나리고 \n나는 나타샤를 생각하고 \n나타샤가 아니올 리 없다 \n언제 벌써 내 속에 고조곤히 와 이야기한다 \n산골로 가는 것은 세상한테 지는 것이 아니다 \n세상 같은 건 더러워 버리는 것이다 \n눈은 푹푹 나리고 \n아름다운 나타샤는 나를 사랑하고 \n어데서 흰 당나귀도 오늘밤이 좋아서 응앙응앙 울을 것이다 '

## 자연어 전처리

> 자연어는 인간의 언어, "사람이 읽기 편하게" 만들어진 언어이다. 대신 거꾸로, 자연어는 컴퓨터가 이해하기 매우 어렵다. 우리는 자연어를 컴퓨터가 이해할 수 있도록 `숫자`의 형태로 변환해 주어야 한다. 

### (1) 기준 세우기

> 어떤 기준으로 우리는 자연어를 숫자의 형태로 묘사할 수 있을까? 

#### 1. 단어를 기준으로 쪼개기

> 공백을 기준으로, 단어를 분리해서 보자

In [ ]:
all_poet = " ".join(poets) # 모든 시를 다 붙여줌
all_poet = all_poet.replace("\n"," ")
words = all_poet.split(" ")

words[:10]

['가난한', '내가', '아름다운', '나타샤를', '사랑해서', '', '오늘밤은', '푹푹', '눈이', '나린다']

In [ ]:
# 각 단어 별 등장 횟수
unique, counts = np.unique(words, return_counts=True) 
word_counts = dict(zip(unique, counts))
print("총 고유한 단어 갯수 : ",len(word_counts))
word_counts 

총 고유한 단어 갯수 :  1024


{'': 59,
 "'도연명'과": 1,
 "'라이넬": 1,
 "'아서라": 1,
 "'프랑시쓰": 1,
 '가': 2,
 '가고': 3,
 '가기를': 1,
 '가까이': 1,
 '가난하고': 2,
 '가난한': 7,
 '가는': 6,
 '가득찬다': 1,
 '가라앉고,': 1,
 '가라앉을': 1,
 '가마타고': 1,
 '가면': 1,
 '가슴가를': 1,
 '가슴은': 1,
 '가슴이': 1,
 '가운데': 1,
 '가을밤같이': 1,
 '가자': 1,
 '가장': 1,
 '가재미의': 1,
 '가지': 1,
 '가지가지': 1,
 '가지취의': 1,
 '가진': 1,
 '간': 2,
 '간다': 1,
 '갈': 1,
 '갈매나무라는': 1,
 '갈바람': 1,
 '감로같은': 1,
 '감주나': 1,
 '갓': 1,
 '갓갓기도': 1,
 '갔다': 2,
 '같고': 3,
 '같구려': 4,
 '같은': 5,
 '같은데': 1,
 '같이': 6,
 '개는': 1,
 '개지': 1,
 '개지꽃에': 1,
 '개포가의': 1,
 '객주': 1,
 '갸웃하는': 1,
 '거닐면': 1,
 '거리': 1,
 '거리를': 2,
 '거리에는': 1,
 '거리엔': 1,
 '건': 1,
 '건너': 1,
 '걸어가는': 2,
 '것': 3,
 '것같이': 1,
 '것과': 4,
 '것도': 1,
 '것들은': 1,
 '것들이다': 1,
 '것만': 6,
 '것만이': 1,
 '것으로': 2,
 '것은': 8,
 '것을': 3,
 '것이': 5,
 '것이다': 8,
 '것이었다.': 3,
 '것인데,': 2,
 '겨울밤': 1,
 '고개를': 1,
 '고기를': 1,
 '고기비눌에': 1,
 '고담하고': 1,
 '고당은': 1,
 '고마운': 1,
 '고무,': 3,
 '고무의': 3,
 '고사리와': 1,
 '고은': 1,
 '고조곤히': 1,
 '골': 1,
 '곰의': 1,
 '곳': 3,
 '곳이다': 1,
 '과부가': 1,
 '괴일': 1,
 '구구한

> 단어를 기준으로 쪼갠 경우, 1000개가 넘는 단어가 나왔다. 이렇게 짧은 시구 속에서는 겹치는 경우가 매우 적다. 그래서 대부분 많은 단어들이 1번 밖에 등장하지 않았다. 전체 데이터 셋을 학습시킬 경우, 1epoch 별로 한번 씩 밖에 학습되지 않고, 그렇기 때문에 오버피팅의 위험이 매우 크다.

#### 2. 음절을 기준으로 쪼개기

> 단어가 아닌 음절을 기준으로 쪼갤 수도 있다.Syllable

In [ ]:
syllables = list(all_poet)

syllables[:15]

['가', '난', '한', ' ', '내', '가', ' ', '아', '름', '다', '운', ' ', '나', '타', '샤']

In [ ]:
# 각 단어 별 등장 횟수
unique, counts = np.unique(syllables, return_counts=True) 
syllable_counts = dict(zip(unique, counts))
print("총 고유한 음절 갯수 : ",len(syllable_counts))
syllable_counts

총 고유한 음절 갯수 :  521


{' ': 1491,
 "'": 8,
 ',': 53,
 '.': 5,
 '가': 109,
 '각': 16,
 '간': 13,
 '갈': 4,
 '감': 4,
 '갓': 3,
 '갔': 2,
 '같': 24,
 '개': 7,
 '객': 1,
 '갸': 1,
 '거': 14,
 '건': 3,
 '걸': 2,
 '겁': 2,
 '것': 58,
 '게': 20,
 '겨': 3,
 '경': 1,
 '계': 2,
 '고': 118,
 '곤': 1,
 '골': 6,
 '곰': 1,
 '곳': 4,
 '공': 1,
 '과': 18,
 '관': 1,
 '괴': 1,
 '구': 21,
 '국': 5,
 '굳': 1,
 '굴': 6,
 '굿': 1,
 '귀': 7,
 '그': 40,
 '근': 1,
 '글': 4,
 '금': 3,
 '급': 1,
 '긋': 2,
 '기': 22,
 '긴': 1,
 '긷': 1,
 '길': 4,
 '김': 3,
 '깊': 3,
 '까': 3,
 '깍': 1,
 '깐': 1,
 '깨': 1,
 '껌': 1,
 '껏': 1,
 '꼬': 1,
 '꼭': 2,
 '꼿': 2,
 '꽃': 5,
 '꽉': 1,
 '꾸': 1,
 '꿇': 1,
 '꿈': 1,
 '꿩': 3,
 '끄': 1,
 '끈': 2,
 '끊': 1,
 '끌': 1,
 '끓': 3,
 '끝': 3,
 '끼': 9,
 '나': 88,
 '난': 12,
 '날': 12,
 '낡': 2,
 '납': 1,
 '났': 3,
 '낮': 3,
 '낯': 2,
 '내': 40,
 '냥': 1,
 '너': 5,
 '넘': 6,
 '네': 1,
 '넥': 1,
 '넬': 1,
 '녀': 8,
 '녁': 6,
 '년': 1,
 '녕': 1,
 '녘': 1,
 '논': 1,
 '놀': 2,
 '높': 5,
 '놓': 3,
 '누': 5,
 '눈': 18,
 '눌': 2,
 '느': 10,
 '는': 141,
 '늘': 4,
 '늙': 3,
 '능': 1,
 '니': 24,
 '닐': 2,
 '다': 82,


> 음절을 기준으로 쪼갠 경우, 500개가 넘는 단어가 나왔다. 단어를 기준으로 쪼갰을 때보다 훨씬 더 겹친다. 하지만 음절을 기준으로 쪼갠 경우에, 시구에 포함되지 않은 음절들은 학습되지 않는다. 예를 들어, '딥러닝'이라는 단어 속의 '딥'은 존재하지 않는다. 이런 음절들을 숫자로 표현할 수 없게 된다. 이렇게 "등장하지 않는 음절"의 이슈는 매우 중요해진다. (사실, 단어로 쪼갰을 때도 이 문제는 동일하게 발생한다.) 이를 해결하기 위해서 우리는 한글을 보다 구성원소 단위로 더 쪼갤 필요가 있다.

In [ ]:
syllable_counts['딥']

KeyError: '딥'

### (3) 초성/중성/종성 분리하기

> 한글은 초성 중성 종성으로 구성된 언어이다.

![](https://img1.daumcdn.net/thumb/R720x0.q80/?scode=mtistory&fname=http%3A%2F%2Fcfile8.uf.tistory.com%2Fimage%2F264A564B53BFED20200B02)

* 초성 : ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ (19개)
* 중성 : ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ (21개)
* 종성 : ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ (27개) - 종성이 없는 경우도 있음

* 조합가능한 모든 글자 수  : (초성 수) \* (중성 수) \* (종성 + 1) = 19 * 21 * 28 = 11,172(개)

> 파이썬에서의 한글은 유니코드라는 체계에서 구성된다. 유니코드에서의 한글은 조합 가능한 모든 글자 수인 11,172개로 구성되어 있는데, "가"~"힣"으로, 유니코드로는 44032(가)~55199(힣)에 순서대로 저장한다.

* Unicode 표

![](http://4.bp.blogspot.com/--xUV4By6kfw/ULXHNtDKSYI/AAAAAAAABac/MbhkZMbfg1g/s1600/unicodehangul.png)

#### 파이썬에서 글자의 유니코드 값 가져오기

In [1]:
ord('가') # 한글 '가'의 유니코드 값

44032

In [2]:
ord('힣') # 한글 '힣'의 유니코드 값

55203

#### 파이썬에서 유니코드 값을 통해 글자를 가져오기

In [ ]:
chr(44032) # 44032의 글자 

'가'

In [ ]:
chr(55203) # 55203의 글자

'힣'

#### 유니코드에서의 음절의 값은 아래의 규칙을 따른다. 

In [ ]:
초성 = (
    u'ㄱ', u'ㄲ', u'ㄴ', u'ㄷ', u'ㄸ', u'ㄹ', u'ㅁ', u'ㅂ', u'ㅃ', u'ㅅ',
    u'ㅆ', u'ㅇ', u'ㅈ', u'ㅉ', u'ㅊ', u'ㅋ', u'ㅌ', u'ㅍ', u'ㅎ'
)

중성 = (
    u'ㅏ', u'ㅐ', u'ㅑ', u'ㅒ', u'ㅓ', u'ㅔ', u'ㅕ', u'ㅖ', u'ㅗ', u'ㅘ',
    u'ㅙ', u'ㅚ', u'ㅛ', u'ㅜ', u'ㅝ', u'ㅞ', u'ㅟ', u'ㅠ', u'ㅡ', u'ㅢ', u'ㅣ'
)

종성 = (
    u'', u'ㄱ', u'ㄲ', u'ㄳ', u'ㄴ', u'ㄵ', u'ㄶ', u'ㄷ', u'ㄹ', u'ㄺ',
    u'ㄻ', u'ㄼ', u'ㄽ', u'ㄾ', u'ㄿ', u'ㅀ', u'ㅁ', u'ㅂ', u'ㅄ', u'ㅅ',
    u'ㅆ', u'ㅇ', u'ㅈ', u'ㅊ', u'ㅋ', u'ㅌ', u'ㅍ', u'ㅎ'
)

1) 초성, 중성, 종성이 주어졌을 때, 합쳐진 음절을 가져오기 

> 특정 음절의 유니코드 값 = $(초성인덱스*21+중성인덱스)*28+종성인덱스+44032$


In [ ]:
# ㄱ + ㅏ + ㄹ -> 갈
value = ((초성.index('ㄱ')*21+중성.index('ㅏ'))*28+종성.index('ㄹ') + 44032)
chr(value)

'갈'

2)  특정 음절이 주어졌을 때, 초성 중성 종성으로 분리하기

* $초성 = ((유니코드값-44032) / 28) / 21$
* $중성 = ((유니코드값-44032) / 28) \% 21$
* $종성 = ((유니코드값-44032) \% 28 $

In [ ]:
# 날 -> ㄴ + ㅏ + ㄹ
value = ord('날')
chosung_index = (value-44032)//28//21
jungsung_index = (value-44032)//28%21
jongsung_index = (value-44032)%28

In [ ]:
print("초성 : ", 초성[chosung_index])
print("중성 : ", 중성[jungsung_index])
print("종성 : ", 종성[jongsung_index])

초성 :  ㄴ
중성 :  ㅏ
종성 :  ㄹ


#### 위와 같이 음절을 자모자로 분리하고, 자모자를 음절로 분리하는 기능을 미리 구현한 것이 있어, 그것을 써보자

code reference : [github-hgtk](https://github.com/bluedisk/hangul-toolkit)

In [ ]:
from korean import compose, decompose

In [ ]:
phrase = all_poet[:22]
phrase

'가난한 내가 아름다운 나타샤를 사랑해서 '

In [ ]:
jamo_phrase = decompose(phrase) # 자모자로 분리하기
jamo_phrase

'ㄱㅏᴥㄴㅏㄴᴥㅎㅏㄴᴥ ᴥㄴㅐᴥㄱㅏᴥ ᴥㅇㅏᴥㄹㅡㅁᴥㄷㅏᴥㅇㅜㄴᴥ ᴥㄴㅏᴥㅌㅏᴥㅅㅑᴥㄹㅡㄹᴥ ᴥㅅㅏᴥㄹㅏㅇᴥㅎㅐᴥㅅㅓᴥ ᴥ'

In [ ]:
phrase = compose(jamo_phrase) # 자모자를 음절로 묶기
phrase

'가난한 내가 아름다운 나타샤를 사랑해서 '

> 각 음절 간 구별을 위해 구별자로서 `ᴥ`를 두었다. 이것이 없으면, 초성과 종성의 구분이 어려워 compose하기가 어렵기 때문이다.

In [ ]:
# 시에 구성된 글자들을 자음, 모음 분리하기
jamo_poets = []
for poet in poets:
    jamo = decompose(poet)
    jamo_poets.append(jamo)
print("poet : \n",poet)
print("jamo : \n",jamo)

poet : 
 명절날 나는 엄매 아배 따라 우리집 개는 나를 따라 
진할머니 진할아버지 있는 큰집으로 가면

얼굴에 별자국이 솜솜 난 말수와 같이 눈도 껌벅거리는 하로에 베 한 필을 
짠다는 벌 하나 건너 집엔 복숭아나무가 많은 신리 고무, 고무의 딸 이녀, 작은 이녀
열여섯에 사십이 넘은 홀아비의 후처가 된, 포족족하니 성이 
잘 나는, 살빛이 매감탕 같은 입술과 젖꼭지는 더 까만 예수쟁이 마을 가까이 
사는 토산 고무, 고무의 딸 승녀, 아들 승동이
육십리라고 해서 파랗게 뵈이는 산을 넘어 있다는 해변에서 과부가 
된 코끝이 빨간 언제나 흰 옷이 정하든, 말 끝에 설게 눈물을 짤 때가 많은 큰
골 고무, 고무의 딸 홍녀, 아들 홍동이, 작은 홍동이
배나무접을 잘하는 주정을 하면 토방돌을 뽑는, 오리치를 잘 놓는, 먼 섬
에 반디젓 담그러 가기를 좋아하는 삼춘, 삼춘 엄매, 사춘 누이, 사춘 동생들
이 그득히들 할머니 할아버지가 안간에들 모여서 방안에서는 새 옷의 내음
새가 나고 
또 인절미, 송구떡, 콩가루차떡의 내음새도 나고, 끼때의 두부와 콩나물과 
뽁운 잔디와 고사리와 도야지비계는 모두 선득선득하니 찬 것들이다

저녁술을 놓은 아이들은 오양간섶 밭마당에 달린 배나무 
동산에서 쥐잡이를 하고 숨굴막질을 하고 꼬리잡이를 하고 
가마타고 시집가는 놀음 말타고 장가가는 놀음을 하고 
이렇게 밤이 어둡도록 북적하니 논다

밤이 깊어 가는 집안엔 엄매는 엄매들끼리 아르간에서들 웃고 이야기하고
아이들은 아이들끼리 웃간 한 방을 잡고 조아질하고 쌈방이 굴리고 바리 깨돌
림하고 호박떼기하고 제비손이구손이하고 이렇게 화디의 사기방 등에 심지를 
몇 번이나 돋우고 홍게닭이 몇 번이나 울어서 졸음이 오면 아릇목싸움 자리싸
움을 하며 히드득거리다 잠이 든다 그래서는 문창에 텅납새의 그림자가 치는 
아츰 시누이 동세들이 육적하니 흥성거리는 부엌으론 샛문틈으로 장지문틈으로 
무이징게 국을 끓이는 맛있는 내음새가 올라오도록 잔다
jamo : 
 ㅁㅕㅇᴥㅈㅓㄹᴥㄴㅏㄹᴥ ᴥㄴㅏᴥㄴ

우리는 자모 분리한 것을 이제 각각 하나의 숫자로 표현해야 한다. 우리가 이전에 배운 one-hot encoding을 통해 아래처럼 구현할 것이다. 

| 자모자 | 번호 | one-hot encoding |
| ---- | --- | --- | 
| ㄱ | 0 | [1, 0, 0, .... ] |
| ㄲ | 1 | [0, 1, 0, .... ] |
| ㄴ | 2 | [0, 0, 1, .... ] |
| ... | ... | .... |


In [ ]:
# 초성과 종성은 겹치는 글자가 존재함으로, 그것을 제외해줌
미포함종성 = tuple(set(종성) - set(초성)) 
jamos = 초성 + 중성 + 미포함종성 + ("ᴥ"," ","\n",) # 초성, 중성, 종성, 그리고 "ᴥ"를 포함
print("독립적인 자모자의 수 : ",len(jamos))
jamos[:10]

독립적인 자모자의 수 :  55


('ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ')

> 위와 같이 한글과 줄 띄어쓰기, 띄어쓰기로만 구성되었기 때문에, 자모자가 아닌 다른 글자들, 예를들어 `숫자`, 혹은 `특수문자`가 포함된 경우에는 그것을 날려주어야 한다. 지금 구현된 `compose`, `decompose`의 메소드는 숫자와 특수문자들을 제거해주는 식으로 구현되어 있다.

In [ ]:
onehot_matrix = np.eye(len(jamos))
onehot_matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
jamo2vec = {} # jamo에 매칭되는 vector
for jamo, onehot_vector in zip(jamos, onehot_matrix):
    jamo2vec[jamo] = onehot_vector
    
print("ㄱ에 매칭된 one-hot vector : \n", jamo2vec['ㄱ'])
print("ㄲ에 매칭된 one-hot vector : \n", jamo2vec['ㄲ'])
print("ㄴ에 매칭된 one-hot vector : \n", jamo2vec['ㄴ'])

ㄱ에 매칭된 one-hot vector : 
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
ㄲ에 매칭된 one-hot vector : 
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
ㄴ에 매칭된 one-hot vector : 
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
jamo_poet = jamo_poets[0]
jamo_poet

'ㄱㅏᴥㄴㅏㄴᴥㅎㅏㄴᴥ ᴥㄴㅐᴥㄱㅏᴥ\nᴥㅇㅏᴥㄹㅡㅁᴥㄷㅏᴥㅇㅜㄴᴥ ᴥㄴㅏᴥㅌㅏᴥㅅㅑᴥㄹㅡㄹᴥ ᴥㅅㅏᴥㄹㅏㅇᴥㅎㅐᴥㅅㅓᴥ ᴥ\nᴥㅇㅗᴥㄴㅡㄹᴥㅂㅏㅁᴥㅇㅡㄴᴥ ᴥㅍㅜㄱᴥㅍㅜㄱᴥ ᴥㄴㅜㄴᴥㅇㅣᴥ ᴥㄴㅏᴥㄹㅣㄴᴥㄷㅏᴥ ᴥ\nᴥㄴㅏᴥㅌㅏᴥㅅㅑᴥㄹㅡㄹᴥ ᴥㅅㅏᴥㄹㅏㅇᴥㅇㅡㄴᴥ ᴥㅎㅏᴥㄱㅗᴥ ᴥ\nᴥㄴㅜㄴᴥㅇㅡㄴᴥ ᴥㅍㅜㄱᴥㅍㅜㄱᴥ ᴥㄴㅏㄹᴥㄹㅣᴥㄱㅗᴥ ᴥ\nᴥㄴㅏᴥㄴㅡㄴᴥ ᴥㅎㅗㄴᴥㅈㅏᴥ ᴥㅆㅡㄹᴥㅆㅡㄹᴥㅎㅣᴥ ᴥㅇㅏㄵᴥㅇㅓᴥ ᴥㅅㅗᴥㅈㅜᴥㄹㅡㄹᴥ ᴥㅁㅏᴥㅅㅣㄴᴥㄷㅏᴥ ᴥ\nᴥㅅㅗᴥㅈㅜᴥㄹㅡㄹᴥ ᴥㅁㅏᴥㅅㅣᴥㅁㅕᴥ ᴥㅅㅐㅇᴥㄱㅏㄱᴥㅎㅏㄴᴥㄷㅏᴥ ᴥ\nᴥㄴㅏᴥㅌㅏᴥㅅㅑᴥㅇㅘᴥ ᴥㄴㅏᴥㄴㅡㄴᴥ ᴥㄴㅜㄴᴥㅇㅣᴥ ᴥㅍㅜㄱᴥㅍㅜㄱᴥ ᴥㅆㅏㅎᴥㅇㅣᴥㄴㅡㄴᴥ ᴥㅂㅏㅁᴥ\nᴥㅎㅢㄴᴥ ᴥㄷㅏㅇᴥㄴㅏᴥㄱㅟᴥ ᴥㅌㅏᴥㄱㅗᴥ ᴥㅅㅏㄴᴥㄱㅗㄹᴥㄹㅗᴥ ᴥㄱㅏᴥㅈㅏᴥ ᴥㅊㅜㄹᴥㅊㅜㄹᴥㅇㅣᴥ ᴥㅇㅜᴥㄴㅡㄴᴥ\nᴥㄱㅣㅍᴥㅇㅡㄴᴥ ᴥㅅㅏㄴᴥㄱㅗㄹᴥㄹㅗᴥ ᴥㄱㅏᴥ ᴥㅁㅏᴥㄱㅏᴥㄹㅣᴥㅇㅔᴥ ᴥㅅㅏㄹᴥㅈㅏᴥ ᴥ\nᴥㄴㅜㄴᴥㅇㅡㄴᴥ ᴥㅍㅜㄱᴥㅍㅜㄱᴥ ᴥㄴㅏᴥㄹㅣᴥㄱㅗᴥ ᴥ\nᴥㄴㅏᴥㄴㅡㄴᴥ ᴥㄴㅏᴥㅌㅏᴥㅅㅑᴥㄹㅡㄹᴥ ᴥㅅㅐㅇᴥㄱㅏㄱᴥㅎㅏᴥㄱㅗᴥ ᴥ\nᴥㄴㅏᴥㅌㅏᴥㅅㅑᴥㄱㅏᴥ ᴥㅇㅏᴥㄴㅣᴥㅇㅗㄹᴥ ᴥㄹㅣᴥ ᴥㅇㅓㅄᴥㄷㅏᴥ ᴥ\nᴥㅇㅓㄴᴥㅈㅔᴥ ᴥㅂㅓㄹᴥㅆㅓᴥ ᴥㄴㅐᴥ ᴥㅅㅗㄱᴥㅇㅔᴥ ᴥㄱㅗᴥㅈㅗᴥㄱㅗㄴᴥㅎㅣᴥ ᴥㅇㅘᴥ ᴥㅇㅣᴥㅇㅑᴥㄱㅣᴥㅎㅏㄴᴥㄷㅏᴥ ᴥ\nᴥㅅㅏㄴᴥㄱㅗㄹᴥㄹㅗᴥ ᴥㄱㅏᴥㄴㅡㄴᴥ ᴥㄱㅓㅅᴥㅇㅡㄴᴥ ᴥㅅㅔᴥㅅㅏㅇᴥㅎㅏㄴᴥㅌㅔᴥ ᴥㅈㅣᴥㄴㅡㄴᴥ ᴥㄱㅓㅅᴥㅇㅣᴥ ᴥㅇㅏᴥㄴㅣᴥㄷㅏᴥ ᴥ\nᴥㅅㅔᴥㅅㅏㅇᴥ ᴥㄱㅏㅌᴥㅇㅡㄴᴥ ᴥㄱㅓㄴᴥ ᴥㄷㅓᴥㄹㅓᴥㅇㅝᴥ ᴥㅂㅓᴥㄹㅣᴥㄴㅡㄴᴥ ᴥㄱㅓㅅᴥㅇㅣᴥㄷㅏᴥ ᴥ\nᴥㄴㅜㄴᴥㅇㅡㄴᴥ ᴥㅍㅜㄱᴥㅍㅜㄱᴥ ᴥㄴㅏᴥㄹㅣᴥㄱㅗᴥ ᴥ\nᴥㅇㅏᴥㄹㅡㅁᴥㄷㅏᴥㅇㅜㄴᴥ ᴥㄴㅏᴥㅌㅏᴥㅅㅑᴥㄴㅡㄴᴥ ᴥㄴㅏᴥㄹㅡㄹᴥ ᴥㅅㅏᴥㄹㅏㅇᴥㅎㅏᴥㄱㅗᴥ ᴥ\nᴥㅇㅓᴥㄷㅔᴥㅅㅓᴥ ᴥㅎㅢㄴᴥ ᴥㄷㅏㅇᴥㄴㅏᴥㄱㅟᴥㄷㅗᴥ ᴥㅇㅗᴥㄴㅡㄹᴥㅂㅏㅁᴥㅇㅣᴥ ᴥㅈㅗㅎᴥㅇㅏᴥㅅㅓᴥ ᴥㅇㅡㅇᴥㅇㅏㅇᴥㅇㅡㅇᴥㅇㅏㅇᴥ ᴥㅇ

### 자모자로 이루어진 전체 글을 벡터화하기

#### 자모자로 된 글을 벡터화하기

In [ ]:
len(jamo_poet)

1002

In [ ]:
jamo_poet

'ㅁㅕㅇᴥㅈㅓㄹᴥㄴㅏㄹᴥ ᴥㄴㅏᴥㄴㅡㄴᴥ ᴥㅇㅓㅁᴥㅁㅐᴥ ᴥㅇㅏᴥㅂㅐᴥ ᴥㄸㅏᴥㄹㅏᴥ ᴥㅇㅜᴥㄹㅣᴥㅈㅣㅂᴥ ᴥㄱㅐᴥㄴㅡㄴᴥ ᴥㄴㅏᴥㄹㅡㄹᴥ ᴥㄸㅏᴥㄹㅏᴥ ᴥ\nᴥㅈㅣㄴᴥㅎㅏㄹᴥㅁㅓᴥㄴㅣᴥ ᴥㅈㅣㄴᴥㅎㅏㄹᴥㅇㅏᴥㅂㅓᴥㅈㅣᴥ ᴥㅇㅣㅆᴥㄴㅡㄴᴥ ᴥㅋㅡㄴᴥㅈㅣㅂᴥㅇㅡᴥㄹㅗᴥ ᴥㄱㅏᴥㅁㅕㄴᴥ\nᴥ\nᴥㅇㅓㄹᴥㄱㅜㄹᴥㅇㅔᴥ ᴥㅂㅕㄹᴥㅈㅏᴥㄱㅜㄱᴥㅇㅣᴥ ᴥㅅㅗㅁᴥㅅㅗㅁᴥ ᴥㄴㅏㄴᴥ ᴥㅁㅏㄹᴥㅅㅜᴥㅇㅘᴥ ᴥㄱㅏㅌᴥㅇㅣᴥ ᴥㄴㅜㄴᴥㄷㅗᴥ ᴥㄲㅓㅁᴥㅂㅓㄱᴥㄱㅓᴥㄹㅣᴥㄴㅡㄴᴥ ᴥㅎㅏᴥㄹㅗᴥㅇㅔᴥ ᴥㅂㅔᴥ ᴥㅎㅏㄴᴥ ᴥㅍㅣㄹᴥㅇㅡㄹᴥ ᴥ\nᴥㅉㅏㄴᴥㄷㅏᴥㄴㅡㄴᴥ ᴥㅂㅓㄹᴥ ᴥㅎㅏᴥㄴㅏᴥ ᴥㄱㅓㄴᴥㄴㅓᴥ ᴥㅈㅣㅂᴥㅇㅔㄴᴥ ᴥㅂㅗㄱᴥㅅㅜㅇᴥㅇㅏᴥㄴㅏᴥㅁㅜᴥㄱㅏᴥ ᴥㅁㅏㄶᴥㅇㅡㄴᴥ ᴥㅅㅣㄴᴥㄹㅣᴥ ᴥㄱㅗᴥㅁㅜᴥ ᴥㄱㅗᴥㅁㅜᴥㅇㅢᴥ ᴥㄸㅏㄹᴥ ᴥㅇㅣᴥㄴㅕᴥ ᴥㅈㅏㄱᴥㅇㅡㄴᴥ ᴥㅇㅣᴥㄴㅕᴥ\nᴥㅇㅕㄹᴥㅇㅕᴥㅅㅓㅅᴥㅇㅔᴥ ᴥㅅㅏᴥㅅㅣㅂᴥㅇㅣᴥ ᴥㄴㅓㅁᴥㅇㅡㄴᴥ ᴥㅎㅗㄹᴥㅇㅏᴥㅂㅣᴥㅇㅢᴥ ᴥㅎㅜᴥㅊㅓᴥㄱㅏᴥ ᴥㄷㅚㄴᴥ ᴥㅍㅗᴥㅈㅗㄱᴥㅈㅗㄱᴥㅎㅏᴥㄴㅣᴥ ᴥㅅㅓㅇᴥㅇㅣᴥ ᴥ\nᴥㅈㅏㄹᴥ ᴥㄴㅏᴥㄴㅡㄴᴥ ᴥㅅㅏㄹᴥㅂㅣㅊᴥㅇㅣᴥ ᴥㅁㅐᴥㄱㅏㅁᴥㅌㅏㅇᴥ ᴥㄱㅏㅌᴥㅇㅡㄴᴥ ᴥㅇㅣㅂᴥㅅㅜㄹᴥㄱㅘᴥ ᴥㅈㅓㅈᴥㄲㅗㄱᴥㅈㅣᴥㄴㅡㄴᴥ ᴥㄷㅓᴥ ᴥㄲㅏᴥㅁㅏㄴᴥ ᴥㅇㅖᴥㅅㅜᴥㅈㅐㅇᴥㅇㅣᴥ ᴥㅁㅏᴥㅇㅡㄹᴥ ᴥㄱㅏᴥㄲㅏᴥㅇㅣᴥ ᴥ\nᴥㅅㅏᴥㄴㅡㄴᴥ ᴥㅌㅗᴥㅅㅏㄴᴥ ᴥㄱㅗᴥㅁㅜᴥ ᴥㄱㅗᴥㅁㅜᴥㅇㅢᴥ ᴥㄸㅏㄹᴥ ᴥㅅㅡㅇᴥㄴㅕᴥ ᴥㅇㅏᴥㄷㅡㄹᴥ ᴥㅅㅡㅇᴥㄷㅗㅇᴥㅇㅣᴥ\nᴥㅇㅠㄱᴥㅅㅣㅂᴥㄹㅣᴥㄹㅏᴥㄱㅗᴥ ᴥㅎㅐᴥㅅㅓᴥ ᴥㅍㅏᴥㄹㅏㅎᴥㄱㅔᴥ ᴥㅂㅚᴥㅇㅣᴥㄴㅡㄴᴥ ᴥㅅㅏㄴᴥㅇㅡㄹᴥ ᴥㄴㅓㅁᴥㅇㅓᴥ ᴥㅇㅣㅆᴥㄷㅏᴥㄴㅡㄴᴥ ᴥㅎㅐᴥㅂㅕㄴᴥㅇㅔᴥㅅㅓᴥ ᴥㄱㅘᴥㅂㅜᴥㄱㅏᴥ ᴥ\nᴥㄷㅚㄴᴥ ᴥㅋㅗᴥㄲㅡㅌᴥㅇㅣᴥ ᴥㅃㅏㄹᴥㄱㅏㄴᴥ ᴥㅇㅓㄴᴥㅈㅔᴥㄴㅏᴥ ᴥㅎㅢㄴᴥ ᴥㅇㅗㅅᴥㅇㅣᴥ ᴥㅈㅓㅇᴥㅎㅏᴥㄷㅡㄴᴥ ᴥㅁㅏㄹᴥ ᴥㄲㅡㅌᴥㅇㅔᴥ ᴥㅅㅓㄹᴥㄱㅔᴥ ᴥㄴㅜㄴᴥㅁㅜㄹᴥㅇㅡㄹᴥ ᴥㅉㅏㄹᴥ ᴥㄸㅐᴥㄱㅏᴥ ᴥㅁㅏㄶᴥㅇㅡㄴᴥ ᴥㅋㅡㄴᴥ\nᴥㄱㅗㄹᴥ ᴥㄱㅗᴥ

In [ ]:
jamo_vectors = np.stack([jamo2vec[char] for char in jamo_poet])
jamo_vectors

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [ ]:
jamo_vectors.shape # 1002자의 자모가 각각 55크기의 벡터로 변환되어 있다

(1002, 55)

In [ ]:
jamo_list = [jamos[np.argmax(vector)] for vector in jamo_vectors]
jamo = "".join(jamo_list)
compose(jamo)

'가난한 내가\n아름다운 나타샤를 사랑해서 \n오늘밤은 푹푹 눈이 나린다 \n나타샤를 사랑은 하고 \n눈은 푹푹 날리고 \n나는 혼자 쓸쓸히 앉어 소주를 마신다 \n소주를 마시며 생각한다 \n나타샤와 나는 눈이 푹푹 쌓이는 밤\n흰 당나귀 타고 산골로 가자 출출이 우는\n깊은 산골로 가 마가리에 살자 \n눈은 푹푹 나리고 \n나는 나타샤를 생각하고 \n나타샤가 아니올 리 없다 \n언제 벌써 내 속에 고조곤히 와 이야기한다 \n산골로 가는 것은 세상한테 지는 것이 아니다 \n세상 같은 건 더러워 버리는 것이다 \n눈은 푹푹 나리고 \n아름다운 나타샤는 나를 사랑하고 \n어데서 흰 당나귀도 오늘밤이 좋아서 응앙응앙 울을 것이다 '

## 우리의 모델은 어떤 식으로 동작해야 할까

![](../../../misc/images/4강_RNN_poet_generator_model.png)

In [ ]:
time_steps = 30 # 몇개의 자모자를 보고 다음 자모자를 예측할지 결정
word_ndims = len(jamos) # 임베딩한 자모자의 갯수

data = []
targets = []
for poet in poets:
    jamo_poet = decompose(poet) # 시의 음절을 자모자로 분리
    jamo_vectors = np.stack([jamo2vec[char] for char in jamo_poet]) # 벡터화한 후 stack

    len_seqs = len(jamo_vectors)
    for i in range(len_seqs-time_steps):
        # 하나씩 지나가면서, 데이터를 확보
        datum = jamo_vectors[i:i+time_steps]
        target = jamo_vectors[i+1:i+1+time_steps]
        data.append(datum)
        targets.append(target)
        
X = np.array(data)
Y = np.array(targets)

In [ ]:
X.shape # 총 16186의 문장, 30개로 이루어진 time step, 55개로 이루어진 word vec

(16186, 30, 55)

In [ ]:
Y.shape # 우리가 예측하려는 문장도 같다

(16186, 30, 55)

### Build RNN Graph

In [ ]:
_, n_steps, n_inputs = X.shape
_, _, n_outputs = Y.shape
n_units = 200
n_layers = 3

graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, [None, n_steps, n_inputs], name='input')
    
    with tf.variable_scope("rnn"):
        cells = [tf.nn.rnn_cell.LSTMCell(n_units)
                 for i in range(n_layers)]
        
        multicell = tf.nn.rnn_cell.MultiRNNCell(cells)
        cell_outputs, state = tf.nn.dynamic_rnn(multicell, x,
                                                 dtype=tf.float32)

    outputs_flat = tf.reshape(cell_outputs, [-1, n_units])
    output = tf.layers.dense(outputs_flat, n_outputs)
        
    # graph에 명시적으로 이름을 추가
    output = tf.identity(output, name="output")
    
    y_true = tf.placeholder(tf.float32, [None, n_steps, n_outputs], name='label')
    flat_y_true = tf.reshape(y_true, [-1, n_outputs])
    
    loss = tf.losses.softmax_cross_entropy(flat_y_true, output)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    
    training_op = optimizer.minimize(loss)

In [ ]:
X_train, X_valid, Y_train, Y_valid =\
train_test_split(X,Y, test_size=0.2)

In [ ]:
from tqdm import tqdm

In [ ]:
def generate_poet(sess, test_string, generate_length=200):
    """ 주어진 첫 소절을 바탕으로, 이어서 쓰는 시
    """
    # 입력 tensor 가져오기
    x = sess.graph.get_tensor_by_name('input:0')
    
    # 출력 tensor 가져오기
    output = sess.graph.get_tensor_by_name('output:0')
    
    jamo_seqs = decompose(test_string)[:time_steps]
    code_seqs = np.stack([jamo2vec[char] for char in jamo_seqs])
    
    for _ in range(generate_length):
        X_batch = np.array(code_seqs[-time_steps:]).reshape(1, time_steps, word_ndims)
        y_pred = sess.run(output, feed_dict={x:X_batch})        
        # 결과 값에서 가장 큰 값을 기준으로 다시 one-hot encoding
        idx = np.argmax(y_pred[-1])
        y_ = np.zeros_like(y_pred[-1])
        y_[idx] = 1
        code_seqs = np.concatenate([code_seqs,y_[np.newaxis]],axis=0)
    generated_poet = compose("".join([jamos[code] for code in np.argmax(code_seqs,axis=1)]))
    return generated_poet


In [ ]:
epochs = 50
batch_size = 20

with graph.as_default():
    sess = tf.Session(graph=graph)
    sess.run(tf.global_variables_initializer())

    n_batches = len(X_train) // batch_size
    for epoch in range(epochs):
        for i in tqdm(range(n_batches)):
            batch_X = X_train[i*batch_size:(i+1)*batch_size]
            batch_Y = Y_train[i*batch_size:(i+1)*batch_size]

            sess.run(training_op,feed_dict={
                x: batch_X, y_true: batch_Y
            })

        val_loss = sess.run(loss, feed_dict={
            x: X_valid, y_true: Y_valid
        })

        print("[epoch : {:2d}] validation loss : [{:.3f}]".format(epoch, val_loss))
        test_input_string = "오늘밤도 아름다운"
        print("test : {} >\n".format(test_input_string))
        print(generate_poet(sess, test_input_string))
        

100%|██████████| 647/647 [00:47<00:00, 13.68it/s]


[epoch :  0] validation loss : [1.529]
test : 오늘밤도 아름다운 >



  0%|          | 2/647 [00:00<00:52, 12.25it/s]

오늘밤도 아름다운 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 아른 


100%|██████████| 647/647 [00:46<00:00, 13.79it/s]


...

[epoch : 18] validation loss : [0.361]
test : 오늘밤도 아름다운 >



  0%|          | 2/647 [00:00<00:50, 12.72it/s]

오늘밤도 아름다운 나타샤는 나를 사랑하고 
어데서 흰 당나귀도 오늘밤이 좋아서 응앙응앙 울을 것이 있었다
산 절의 마당귀에 여인의 머


100%|██████████| 647/647 [00:44<00:00, 14.40it/s]


[epoch : 19] validation loss : [0.359]
test : 오늘밤도 아름다운 >

오늘밤도 아름다운 나타샤는 나를 사랑하고 
어데서 흰 당나귀도 오늘밤이 좋아서 응앙응앙 울을 것이 있어서 나를 마음대로 굴
려 가는 것
